## 로컬 환경에서 PDF 검색하기 2단계 step5
- PDF 문서 여러개 로드하는 방식 추가 (문서 3개 로드)
- PDF 문서를 로드하고 한국어 임베딩 모델을 사용하여 임베딩 데이터 생성
- 임베딩 데이터를 csv 파일로 만들어 저장하기 
- 저장한 csv 데이터를 읽어서 FAISS 인덱스 생성하기
- 생성한 FAISS 인덱스를 검색하기 

- 생성한 FAISS 인덱스에 langchain 프레임워크 적용하여 llm 검색하기 

### - 사용한 임베딩 모델 jhgan/ko-sroberta-multitask
### - 사용한 LLM 모델 llama3.2

In [1]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import DirectoryLoader,PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  
from sentence_transformers import SentenceTransformer

# Step 1: 문서 로드
# 'data' 폴더 내의 모든 pdf 로드

loader = DirectoryLoader(
    'data',
    glob='*.pdf',
    loader_cls=PyMuPDFLoader
)

# pdf_file_path = "data/"
# pdf_file_name = "AI기반_인파분석플랫폼구축_제안서"

# loader = PyMuPDFLoader(pdf_file_path + pdf_file_name + ".pdf") 
docs = loader.load()

# Step 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# Step 3: SentenceTransformer 모델을 LangChain의 Embeddings 클래스로 감싸기
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        """문서 리스트를 벡터로 변환"""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """검색 쿼리를 벡터로 변환"""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


# Step 4: 모델 로드 및 FAISS 인덱스 생성
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")
# faiss_index = FAISS.from_documents(split_documents, embedding_model)

import csv
import pandas as pd
import os

# Step 5: 문서 임베딩 및 CSV 저장
# 원본 문서와 임베딩 데이터를 CSV 에 함께 저장함
# 원본 문서를 저장하는 이유 : 검색 결과를 보여줘야 하기 때문. 
# Step 5: 개별 파일별 CSV 저장 함수
def save_embeddings_to_csv(documents, embedding_model, file_path="./csv/"):
    os.makedirs(file_path, exist_ok=True)

    # 문서별로 그룹화
    file_docs = {}
    for doc in documents:
        file_name = os.path.basename(doc.metadata['source']).replace('.pdf', '')  # 원본 파일 이름 추출
        if file_name not in file_docs:
            file_docs[file_name] = []
        file_docs[file_name].append(doc)

    # 각 파일마다 개별 CSV 저장
    for file_name, docs in file_docs.items():
        full_path = os.path.join(file_path, f"{file_name}.csv")

        # 문서 임베딩 수행
        embeddings = embedding_model.embed_documents([doc.page_content for doc in docs])

        # CSV 저장
        with open(full_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["document", "embedding"])
            
            for doc, embedding in zip(docs, embeddings):
                writer.writerow([doc.page_content, embedding])
        
        print(f"임베딩 데이터가 {full_path} 파일에 저장되었습니다.")

# 함수 실행
documents = split_documents  # FAISS에 넣은 문서 리스트 사용
full_path = save_embeddings_to_csv(documents, embedding_model)

import faiss
import numpy as np
import pandas as pd


# CSV 파일 불러오기 
def load_embeddings_from_csv(filepath):
    df = pd.read_csv(filepath)
    df["embedding"] = df["embedding"].apply(lambda x: np.fromstring(x[1:-1], sep=','))  # 문자열을 numpy 배열로 변환
    return df

# FAISS 인덱스 생성
def create_faiss_index(embedding_dim, df):
    index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스
    embeddings = np.vstack(df["embedding"].values).astype("float32")
    index.add(embeddings)  
    return index, df

# CSV에서 데이터 불러오기
df_embeddings = load_embeddings_from_csv(full_path)

# FAISS 인덱스 생성
embedding_dim = len(df_embeddings["embedding"].iloc[0])  # 벡터 차원 수 확인
faiss_index, df_embeddings = create_faiss_index(embedding_dim, df_embeddings)

print("FAISS 인덱스가 성공적으로 생성되었습니다!")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


임베딩 데이터가 ./csv/SPRI_AI_Brief_2023년12월호_F.csv 파일에 저장되었습니다.
임베딩 데이터가 ./csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.
임베딩 데이터가 ./csv/운영체제_중간과제물.csv 파일에 저장되었습니다.


ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [6]:
# Step 6 : 프롬프트 생성 
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean,and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)

In [10]:
# Step 9 : FAISS 검색 함수 추가
def retrieve_context(query, faiss_index, df, embedding_model, top_k=5):
    """FAISS 인덱스를 사용하여 쿼리와 가장 유사한 문서 검색"""
    query_embedding = np.array(embedding_model.embed_query(query)).astype("float32").reshape(1, -1)
    
    distances, indices = faiss_index.search(query_embedding, top_k)
    
    retrieved_docs = [df.iloc[idx]["document"] for idx in indices[0] if idx < len(df)]
    
    return "\n".join(retrieved_docs)

from langchain_ollama import OllamaLLM

# LLM 및 프롬프트 설정
llm = OllamaLLM(model="llama3.2")
prompt = PromptTemplate.from_template("Context: {context}\nQuestion: {question}")
# Step 10 : 체인 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

# RunnablePassthrough은 입력을 그대로 전달하는 역할
retrieval_chain = (
    {"context": lambda query: retrieve_context(query, faiss_index, df_embeddings, embedding_model), 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Step 11 : 실행 예시
query = "인파탐지 시스템을 설명해주세요"
response = retrieval_chain.invoke(query)
print(response)


인파 탐지 hệ thống(Infrastucture System)은 관제中心에서 운영되는 장비와 시스템으로, 인파 현황 monitoring 및 대시 보드를 통해 사람들이 많이 이동하는 지역이나 시간대에 대한 분석과 대응能力이 있는 시스템입니다. 이 시스템은 다양한 인프라 및 장비를 포함하여 지능형 영상分析기, IoT 스캐너, 지능형 카메라 등으로 구성되어 있습니다.

인파 탐지 시스템의 주요 기능은 다음과 같습니다.

1.  **영상 모니터링**: 인파 현황 monitoring를 위해 사용되는 영상 분석이 가능한 장비입니다.
2.  **디지털 트윈**: 여러 기기와 시스템을 연결하여 원하는 정보를 얻는 데 필요한 디지털 데이터를 공유합니다.
3.  **인파현황 모니터링**: 인파 현황 monitoring에 사용되는 API를 통해 지역 및 시간대-specific의 인파 현황과 혼잡도 분석이 가능합니다.
4.  **대시 보드**: 다양한 인프라와 시스템을 연결하여 원하는 정보를 얻는 데 필요한 디지털 데이터를 공유합니다.
5.  **iot 스캐너**: IoT 스캐너는 IoT 장치를 통해 인파 현황 monitoring에 사용되는 API를 통해 인파 현황과 혼잡도 분석이 가능합니다.

인파 탐지 시스템의 목적은 다음과 같습니다.

1.  **인파 현황 monitoring**: people가 많이 이동하는 지역이나 시간대에 대한 분석과 대응 ability이 있는 시스템입니다.
2.  **혼잡도 분석**: 밀집도, 혼잡도, 이동동선, 체류시간 etc. etc. information을 수집하여 analyze하고 analysis result를 display합니다.

인파 탐지 시스템의 예를 들어 인파탐지 시스템의 특성은 다음과 같습니다.

1.  **대시 보드**: 다양한 인프라와 시스템을 연결하여 원하는 정보를 얻는 데 필요한 디지털 데이터를 공유합니다.
2.  **영상 모니터링**: 인파 현황 monitoring를 위해 사용되는 영상 분석이 가능한 장비입니다.
3.  *